In [1]:
%matplotlib qt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from copy import copy

from gauss import Gaussian
from velocity_motion_model import vmm
from Estimator_EKF import EKF
from Estimator_UKF import UKF
from Estimator_Bayes_Taylor import BayesTaylor
from Error_ellipse import e_ellipse

dt = 1
controls = np.array(np.zeros((2,10)))
wt = (2*np.pi)/10
R = np.array([[0.05,0,0],[0,0.05,0],[0,0,0.02]])
Q = np.array([[0.05,0,0],[0,0.05,0],[0,0,0.02]])

for i in range(0, len(controls[0])):
    controls[0, i] = np.random.normal(5, 1)
    controls[1, i] = np.random.normal(wt,0)    

xt, zt = vmm.sim_move(controls, dt, R, Q)   

In [2]:
##UKF##
fig0 = plt.figure(0)
ax0 = plt.subplot(111, aspect = 'equal')

myUKF = UKF(R, Q, ax0)

estMeans = []
estSigs = []

estMeans.append(np.array([0,0,0]))
estSigs.append(np.array([[0.1, 0, 0],[0, 0.1, 0],[0, 0, 0.1]]))

e_ellipse.plot_eEllipse(estMeans[0], estSigs[0], ax0, col = "black")

for i in range (0, len(controls[0])):
    #print(estimatedMeans[:, i])
    newMean, newSigma = myUKF.func_UKF(estMeans[i], estSigs[i], controls[:, i], zt[:, i+1])
    estMeans.append(newMean)
    estSigs.append(newSigma)


for i in range (0, len(estMeans)):
    #e_ellipse.plot_eEllipse(estMeans[i], estSigs[i], ax1, col = "black")
    drawGauss = Gaussian(["x", "y", "theta"], estMeans[i], estSigs[i])
    drawGauss.marginalizeIndicesUpdate(arrIndices = np.array([2]))
    drawGauss.updateCov()
    #drawGauss.toStringCov()
    e_ellipse.plot_eEllipse(drawGauss.mean, drawGauss.cov, ax0, col = "blue")

#e_ellipse.plot_eEllipse(estMeans, estSigs, ax0)

plt.scatter(xt[0,:], xt[1,:], c = "red", marker = "x")
#plt.scatter(zt[0,:], zt[1,:], c = "blue", marker = ".")

i = 0
for x, y in zip(xt[0,:], xt[1,:]):
    plt.text(x, y, str(i), color="black", fontsize=12)
    i =i+1
plt.margins(0.1)
#    plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()

In [3]:
##EKF##
myEKF = EKF(R, Q)

estMeans = []
estSigs = []

estMeans.append(np.array([0,0,0]))
estSigs.append(np.array([[0.1, 0, 0],[0, 0.1, 0], [0, 0, 0.1]]))

for i in range (0, len(controls[0])):
    newMean, newSigma = myEKF.func_EKF(estMeans[i], estSigs[i], controls[:, i], zt[:, i+1])
    estMeans.append(newMean)
    estSigs.append(newSigma)

for i in range (0, len(estMeans)):
    #e_ellipse.plot_eEllipse(estMeans[i], estSigs[i], ax1, col = "black")
    drawGauss = Gaussian(["x", "y", "theta"], estMeans[i], estSigs[i])
    drawGauss.marginalizeIndicesUpdate(arrIndices = np.array([2]))
    drawGauss.updateCov()
    #drawGauss.toStringCov()
    e_ellipse.plot_eEllipse(drawGauss.mean, drawGauss.cov, ax0, col = "green")

plt.scatter(xt[0,:], xt[1,:], c = "red", marker = "x")
#plt.scatter(zt[0,:], zt[1,:], c = "blue", marker = ".")

i = 0
for x, y in zip(xt[0,:], xt[1,:]):
    plt.text(x, y, str(i), color="black", fontsize=12)
    i =i+1
plt.margins(0.1)
#    plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()

In [4]:
# ##bayes_filter using taylor linearisation##

# beliefs = []
# mean_init = np.array([0,0,0])
# cov_init = np.array([[0.1, 0, 0],[0, 0.1, 0], [0, 0, 0.1]])
# belief_init = Gaussian(['x','y','theta'], mean_init, cov_init)
# beliefs.append(belief_init)

# bayesFilter = BayesTaylor(R, Q)

# for i in range (0, len(controls[0])):
#     myDist = bayesFilter.estimate(beliefs[i], controls[:, i], zt[:, i+1])
#     myDist.updateCov()
#     beliefs.append(myDist)

    
# for i in range (0, len(beliefs)):
#     print(i)
#     #e_ellipse.plot_eEllipse(estMeans[i], estSigs[i], ax1, col = "black")    
#     beliefs[i].marginalizeIndicesUpdate(arrIndices = np.array([2]))
#     beliefs[i].updateCov()
#     print("Cov")
#     beliefs[i].toStringCov()
#     #plt.scatter(beliefs[i].mean[0],beliefs[i].mean[1], c = "red")
#     e_ellipse.plot_eEllipse(beliefs[i].mean, beliefs[i].cov, ax0, col = "orange")

# #plt.scatter(xt[0,:], xt[1,:], c = "black", marker = "x")